In [1]:
import os
import pandas as pd
from datetime import datetime, timezone

In [2]:
def procesar_a_silver(nombre_archivo="donantes_bronze.parquet"):
    """
    Procesa los datos desde Bronze hacia Silver con pivot mensual.
    Calcula totales acumulados y transacciones efectivas (>0).
    Retorna el DataFrame pivot y un resumen mensual consistente.
    """

    # -------------------------------
    # CONFIGURACIÓN
    # -------------------------------
    #base_dir = os.path.dirname(os.path.abspath(__file__))
    base_dir = os.getcwd()
    carpeta_bronze = os.path.join(base_dir, "..", "layer", "bronze")
    carpeta_silver = os.path.join(base_dir, "..", "layer", "silver")
    ruta_bronze = os.path.join(carpeta_bronze, nombre_archivo)

    os.makedirs(carpeta_silver, exist_ok=True)

    # -------------------------------
    # 1. VALIDAR EXISTENCIA DEL ARCHIVO
    # -------------------------------
    if not os.path.exists(ruta_bronze):
        raise FileNotFoundError(f"No se encontró el archivo en Bronze: {ruta_bronze}")
    print(f"✓ Archivo encontrado en Bronze: {ruta_bronze}")

    # -------------------------------
    # 2. LECTURA DEL ARCHIVO PARQUET
    # -------------------------------
    df_silver = pd.read_parquet(ruta_bronze)
    print(f"✓ Archivo leído correctamente. Registros cargados: {len(df_silver)}")

    # ----------------------------------------------------------
    # 3. TRANSFORMACIÓN - ELIMINAR ID QUE NUNCA DONARON Y PIVOT
    # ----------------------------------------------------------
    fecha_pago_none = df_silver[df_silver['Fecha_Pago'].isna()]
    df_silver = df_silver.drop(fecha_pago_none.index)
    df_pivot_silver = df_silver.pivot_table(
        index=['Id_donante', 'Método_Pago', 'Estrategia', 'Status_Socio', 'Año_Mes_Creacion'],
        columns='Año_Mes_Donacion',
        values='Monto_Donacion',
        aggfunc='sum',
        fill_value=0
    ).reset_index()

    # Quitar nombre de columnas jerárquicas
    df_pivot_silver.columns.name = None
    df_pivot_silver = df_pivot_silver.sort_values(by='Id_donante').reset_index(drop=True)

    # -------------------------------
    # 3b. PRINT DE INSPECCIÓN
    # -------------------------------
    print("\n--- Primeros 10 registros de df_pivot_silver ---")
    print(df_pivot_silver.head(10))
    print("\n--- Columnas y tipos ---")
    print(df_pivot_silver.dtypes)

    # -------------------------------
    # 4. GUARDAR PARQUET EN SILVER
    # -------------------------------
    ruta_salida = os.path.join(carpeta_silver, "donantes_silver.parquet")
    ruta_salida_pivot = os.path.join(carpeta_silver, "donantes_silver_pivot.parquet")
    df_silver.to_parquet(ruta_salida, index = False)
    df_pivot_silver.to_parquet(ruta_salida_pivot, index = False)
    print(f"\n✓ Datos procesados y guardados en: {ruta_salida}")
    print(f"\n✓ Datos procesados y guardados en: {ruta_salida_pivot}")

    # -------------------------------
    # 5. ARCHIVO INDICADOR
    # -------------------------------
    indicador_py = os.path.join(carpeta_silver, "donantes_silver.py")
    ahora_utc = datetime.now(timezone.utc)
    with open(indicador_py, "w", encoding="utf-8") as f:
        f.write("# Archivo indicador para la capa Silver\n")
        f.write(f"# Generado: {ahora_utc.isoformat()}\n")
        f.write("# Contiene: donantes_silver.parquet (datos limpios y pivot mensuales)\n")
    print(f"✓ Archivo indicador creado: {indicador_py}")

    # -------------------------------
    # 6. RESUMEN MENSUAL
    # -------------------------------
    meses = [c for c in df_pivot_silver.columns if c not in ['Id_donante', 'Método_Pago', 'Estrategia', 'Status_Socio', 'Año_Mes_Creacion']]
    resumen_mensual = pd.DataFrame(index=meses)
    resumen_mensual['Total_Donaciones'] = df_pivot_silver[meses].sum()
    resumen_mensual['Cantidad_Donaciones_Exitosas'] = (df_pivot_silver[meses] > 0).sum()
    resumen_mensual['Tasa_Exito_%'] = 100  # Silver no tiene info de fallos de cobro, solo pivot positivo

    print("\n--- RESUMEN MENSUAL EN SILVER ---")
    print(resumen_mensual)

    # -------------------------------
    # 7. TOTALES ACUMULADOS
    # -------------------------------
    total_donaciones = resumen_mensual['Total_Donaciones'].sum()
    total_registros = len(df_silver)
    total_transacciones = resumen_mensual['Cantidad_Donaciones_Exitosas'].sum()

    print("\n--- TOTALES ACUMULADOS EN SILVER ---")
    print(f"Suma total Monto_Donacion: {total_donaciones}")
    print(f"\nCantidad total de registros generados: {total_registros}")
    print(f"Total transacciones (>0): {total_transacciones}")

    return resumen_mensual

In [3]:
#if __name__ == "__main__":
#    procesar_a_silver()

procesar_a_silver()

✓ Archivo encontrado en Bronze: C:\Users\germa\Desktop\Carpetas\Data_Engineer_Specialist\Lifetime_Donantes_Medallon\scripts\..\layer\bronze\donantes_bronze.parquet
✓ Archivo leído correctamente. Registros cargados: 258764

--- Primeros 10 registros de df_pivot_silver ---
  Id_donante      Método_Pago    Estrategia Status_Socio Año_Mes_Creacion  \
0    D000001  Tarjeta Crédito  Face to Face       Activo          2023-06   
1    D000002  Tarjeta Crédito  Face to Face       Fugado          2023-06   
2    D000003  Tarjeta Crédito  Face to Face       Fugado          2023-06   
3    D000004  Tarjeta Crédito  Face to Face       Activo          2023-06   
4    D000005       Cuenta Rut  Face to Face       Activo          2023-06   
5    D000006     Cuenta Vista  Face to Face       Activo          2023-06   
6    D000007       Cuenta Rut  Face to Face       Activo          2023-06   
7    D000008       Cuenta Rut  Face to Face       Activo          2023-06   
8    D000009  Tarjeta Crédito  Face

,Total_Donaciones,Cantidad_Donaciones_Exitosas,Tasa_Exito_%
2023-06,8113000.0,774,100
2023-07,15877000.0,1526,100
2023-08,22843000.0,2229,100
2023-09,30346000.0,2959,100
2023-10,37297000.0,3622,100
2023-11,44653000.0,4334,100
2023-12,51488000.0,5052,100
2024-01,58333000.0,5714,100
2024-02,65224000.0,6401,100
2024-03,70689000.0,6922,100


In [4]:
base_dir = os.getcwd()
carpeta_bronze = os.path.join(base_dir, "..", "layer", "bronze")
carpeta_silver = os.path.join(base_dir, "..", "layer", "silver")
ruta_bronze = os.path.join(carpeta_bronze, "donantes_bronze.parquet")

os.makedirs(carpeta_silver, exist_ok=True)

# -------------------------------
# 1. VALIDAR EXISTENCIA DEL ARCHIVO
# -------------------------------
if not os.path.exists(ruta_bronze):
    raise FileNotFoundError(f"No se encontró el archivo en Bronze: {ruta_bronze}")
print(f"✓ Archivo encontrado en Bronze: {ruta_bronze}")

# -------------------------------
# 2. LECTURA DEL ARCHIVO PARQUET
# -------------------------------
df_silver = pd.read_parquet(ruta_bronze)
print(f"✓ Archivo leído correctamente. Registros cargados: {len(df_silver)}")



✓ Archivo encontrado en Bronze: C:\Users\germa\Desktop\Carpetas\Data_Engineer_Specialist\Lifetime_Donantes_Medallon\scripts\..\layer\bronze\donantes_bronze.parquet
✓ Archivo leído correctamente. Registros cargados: 258764


In [5]:
df_silver.count()

Id_donante          258764
Método_Pago         258764
Estrategia          258764
Fecha_Creacion      258764
Fecha_Pago          258129
Monto_Donacion      258129
Status_Socio        258764
Fecha_Fuga           41451
Año_Mes_Creacion    258764
Año_Mes_Donacion    258764
Año_Mes_Fuga        258764
dtype: int64

In [6]:
df_silver.head()

,Id_donante,Método_Pago,Estrategia,Fecha_Creacion,Fecha_Pago,Monto_Donacion,Status_Socio,Fecha_Fuga,Año_Mes_Creacion,Año_Mes_Donacion,Año_Mes_Fuga
0,D000006,Cuenta Vista,Face to Face,2023-06-30,2023-06-01,10000.0,Activo,None,2023-06,2023-06,NaT
1,D000034,Cuenta Vista,Face to Face,2023-06-30,2023-06-01,8000.0,Activo,None,2023-06,2023-06,NaT
2,D000061,Cuenta Vista,Face to Face,2023-06-30,2023-06-01,10000.0,Activo,None,2023-06,2023-06,NaT
3,D000084,Cuenta Rut,Face to Face,2023-06-30,2023-06-01,9000.0,Activo,None,2023-06,2023-06,NaT
4,D000119,Cuenta Rut,Face to Face,2023-06-30,2023-06-01,8000.0,Fugado,2024-02-23,2023-06,2023-06,2024-02


In [7]:
df_silver.shape

(258764, 11)

In [8]:

fecha_pago_none

NameError: name 'fecha_pago_none' is not defined

In [ ]:
df_silver = df_silver.drop(fecha_pago_none.index)
df_silver.shape

In [ ]:
df_silver_89 = df_silver[df_silver['Id_donante'] == "D000089"]
df_silver_89


In [ ]:
# -------------------------------
# 3. TRANSFORMACIÓN PIVOT
# -------------------------------
df_pivot_silver = df_silver.pivot_table(
    index=['Id_donante', 'Método_Pago', 'Estrategia', 'Status_Socio', 'Año_Mes_Creacion'],
    columns='Año_Mes_Donacion',
    values='Monto_Donacion',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Quitar nombre de columnas jerárquicas
df_pivot_silver.columns.name = None
df_pivot_silver = df_pivot_silver.sort_values(by='Id_donante').reset_index(drop=True)

# -------------------------------
# 3b. PRINT DE INSPECCIÓN
# -------------------------------
print("\n--- Primeros 10 registros de df_pivot_silver ---")
print(df_pivot_silver.head(10))
print("\n--- Columnas y tipos ---")
print(df_pivot_silver.dtypes)

# -------------------------------
# 4. GUARDAR PARQUET EN SILVER
# -------------------------------
ruta_salida = os.path.join(carpeta_silver, "donantes_silver.parquet")
df_pivot_silver.to_parquet(ruta_salida, index=False)
print(f"\n✓ Datos procesados y guardados en: {ruta_salida}")

# -------------------------------
# 5. ARCHIVO INDICADOR
# -------------------------------
indicador_py = os.path.join(carpeta_silver, "donantes_silver.py")
ahora_utc = datetime.now(timezone.utc)
with open(indicador_py, "w", encoding="utf-8") as f:
    f.write("# Archivo indicador para la capa Silver\n")
    f.write(f"# Generado: {ahora_utc.isoformat()}\n")
    f.write("# Contiene: donantes_silver.parquet (datos limpios y pivot mensuales)\n")
print(f"✓ Archivo indicador creado: {indicador_py}")

# -------------------------------
# 6. RESUMEN MENSUAL
# -------------------------------
meses = [c for c in df_pivot_silver.columns if c not in ['Id_donante', 'Método_Pago', 'Estrategia', 'Status_Socio', 'Año_Mes_Creacion']]
resumen_mensual = pd.DataFrame(index=meses)
resumen_mensual['Total_Donaciones'] = df_pivot_silver[meses].sum()
resumen_mensual['Cantidad_Donaciones_Exitosas'] = (df_pivot_silver[meses] > 0).sum()
resumen_mensual['Tasa_Exito_%'] = 100  # Silver no tiene info de fallos de cobro, solo pivot positivo

print("\n--- RESUMEN MENSUAL EN SILVER ---")
print(resumen_mensual)

# -------------------------------
# 7. TOTALES ACUMULADOS
# -------------------------------
total_donaciones = resumen_mensual['Total_Donaciones'].sum()
total_registros = len(df_silver)
total_transacciones = resumen_mensual['Cantidad_Donaciones_Exitosas'].sum()

print("\n--- TOTALES ACUMULADOS EN SILVER ---")
print(f"Suma total Monto_Donacion: {total_donaciones}")
print(f"\nCantidad total de registros generados: {total_registros}")
print(f"Total transacciones (>0): {total_transacciones}")

return resumen_mensual